In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
path = 'individual-datasets/drought-data/'

In [2]:
counties = os.listdir(path)
remove = []
for i in range(len(counties)):
    if '.csv' in counties[i]:
        counties[i] = counties[i][:-4].title()
    else:
        remove.append(counties[i])

for r in remove:
    counties.remove(r)

In [3]:
files = os.listdir(path)
drought_dfs = {}
for f in files:
    if '.csv' in f:
        drought_dfs[f[:-4].title()] = pd.read_csv(path+f)

In [4]:
final_df = pd.DataFrame()
for county, df in drought_dfs.items():
    df = df[['Week', 'DSCI']]
    df.Week = pd.to_datetime(df.Week)
    df = pd.DataFrame(df.groupby(df.Week.dt.year)['DSCI'])
    df[1] = df[1].apply(lambda x: x.mean())
    df.columns = ['Year', county]
    df = df.set_index('Year')
    if len(final_df) > 0:
        final_df = final_df.join(df, on=['Year'], how='left')
    else:
        final_df = df.copy()
final_df = pd.melt(final_df.reset_index(), id_vars = ['Year'], value_vars = counties)
final_df = final_df.rename({'value': 'DSCI', 'variable':'County'}, axis=1)
final_df

/var/folders/49/b4fjywl16vqb_14cy97qv8s40000gn/T/ipykernel_67137/4218427996.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Week = pd.to_datetime(df.Week)
/var/folders/49/b4fjywl16vqb_14cy97qv8s40000gn/T/ipykernel_67137/4218427996.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Week = pd.to_datetime(df.Week)
/var/folders/49/b4fjywl16vqb_14cy97qv8s40000gn/T/ipykernel_67137/4218427996.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,Year,County,DSCI
0,2000,Franklin,59.461538
1,2001,Franklin,125.730769
2,2002,Franklin,307.433962
3,2003,Franklin,0.000000
4,2004,Franklin,0.000000
...,...,...,...
2370,2020,Isle Of Wight,0.826923
2371,2021,Isle Of Wight,44.211538
2372,2022,Isle Of Wight,24.057692
2373,2023,Isle Of Wight,26.461538


In [5]:
fips_map = {
    '0001': 'Accomack',
    '0003': 'Albemarle',
    '0510': 'Alexandria City',
    '0005': 'Alleghany',
    '0007': 'Amelia',
    '0009': 'Amherst',
    '0011': 'Appomattox',
    '0013': 'Arlington',
    '0015': 'Augusta',
    '0017': 'Bath',
    '0019': 'Bedford',
    '0021': 'Bland',
    '0023': 'Botetourt',
    '0520': 'Bristol City',
    '0025': 'Brunswick',
    '0027': 'Buchanan',
    '0029': 'Buckingham',
    '0530': 'Buena Vista City',
    '0031': 'Campbell',
    '0033': 'Caroline',
    '0035': 'Carroll',
    '0036': 'Charles City',
    '0037': 'Charlotte',
    '0540': 'Charlottesville City',
    '0550': 'Chesapeake City',
    '0041': 'Chesterfield',
    '0043': 'Clarke',
    '0570': 'Colonial Heights City',
    '0580': 'Covington City',
    '0045': 'Craig',
    '0047': 'Culpeper',
    '0049': 'Cumberland',
    '0590': 'Danville City',
    '0051': 'Dickenson',
    '0053': 'Dinwiddie',
    '0057': 'Essex',
    '0059': 'Fairfax',
    '0061': 'Fauquier',
    '0063': 'Floyd',
    '0065': 'Fluvanna',
    '0067': 'Franklin',
    '0620': 'Franklin City',
    '0069': 'Frederick',
    '0630': 'Fredericksburg City',
    '0640': 'Galax City',
    '0071': 'Giles',
    '0073': 'Gloucester',
    '0075': 'Goochland',
    '0077': 'Grayson',
    '0079': 'Greene',
    '0081': 'Greensville',
    '0083': 'Halifax',
    '0650': 'Hampton City',
    '0085': 'Hanover',
    '0660': 'Harrisonburg City',
    '0087': 'Henrico',
    '0089': 'Henry',
    '0091': 'Highland',
    '0670': 'Hopewell City',
    '0093': 'Isle Of Wight',
    '0095': 'James City',
    '0097': 'King And Queen',
    '0099': 'King George',
    '0101': 'King William',
    '0103': 'Lancaster',
    '0105': 'Lee',
    '0678': 'Lexington City',
    '0109': 'Louisa',
    '0107': 'Loudoun',
    '0111': 'Lunenburg',
    '0113': 'Madison',
    '0683': 'Manassas City',
    '0685': 'Manassas Park City',
    '0690': 'Martinsville City',
    '0115': 'Mathews',
    '0117': 'Mecklenburg',
    '0119': 'Middlesex',
    '0121': 'Montgomery',
    '0125': 'Nelson',
    '0127': 'New Kent',
    '0700': 'Newport News City',
    '0710': 'Norfolk City',
    '0131': 'Northampton',
    '0133': 'Northumberland',
    '0720': 'Norton City',
    '0135': 'Nottoway',
    '0137': 'Orange',
    '0139': 'Page',
    '0141': 'Patrick',
    '0730': 'Petersburg City',
    '0143': 'Pittsylvania',
    '0740': 'Portsmouth City',
    '0145': 'Powhatan',
    '0147': 'Prince Edward',
    '0149': 'Prince George',
    '0153': 'Prince William',
    '0155': 'Pulaski',
    '0750': 'Radford City',
    '0157': 'Rappahannock',
    '0159': 'Richmond',
    '0760': 'Richmond City',
    '0161': 'Roanoke',
    '0770': 'Roanoke City',
    '0163': 'Rockbridge',
    '0165': 'Rockingham',
    '0167': 'Russell',
    '0169': 'Scott',
    '0171': 'Shenandoah',
    '0173': 'Smyth',
    '0175': 'Southampton',
    '0177': 'Spotsylvania',
    '0179': 'Stafford',
    '0790': 'Staunton City',
    '0800': 'Suffolk City',
    '0181': 'Surry',
    '0183': 'Sussex',
    '0185': 'Tazewell',
    '0810': 'Virginia Beach City',
    '0187': 'Warren',
    '0191': 'Washington',
    '0820': 'Waynesboro City',
    '0193': 'Westmoreland',
    '0830': 'Williamsburg City',
    '0840': 'Winchester City',
    '0195': 'Wise',
    '0197': 'Wythe',
    '0199': 'York'
}

fips_map = {v: k for k, v in fips_map.items()}

In [6]:
final_df['County'] = ['51'+fips_map[county][-3:] for county in final_df.County]

In [7]:
len(counties)

95

In [8]:
final_df = final_df[(final_df['Year'] >= 2008) & (final_df['Year'] <= 2022)]

In [9]:
final_df

,Year,County,DSCI
8,2008,51067,181.981132
9,2009,51067,27.865385
10,2010,51067,27.653846
11,2011,51067,50.346154
12,2012,51067,62.115385
...,...,...,...
2368,2018,51093,16.884615
2369,2019,51093,11.094340
2370,2020,51093,0.826923
2371,2021,51093,44.211538


In [10]:
path = 'individual-datasets/'

In [22]:
crops = pd.read_csv(path+'crop_data.csv')
crops

,value,count,year,county,Crop
0,1,140068.0,2008,51001,Corn
1,2,229.0,2008,51001,Cotton
2,4,120.0,2008,51001,Sorghum
3,5,128850.0,2008,51001,Soybeans
4,6,88.0,2008,51001,Sunflower
...,...,...,...,...,...
49909,190,48862.0,2022,51199,Woody_Wetlands
49910,195,8694.0,2022,51199,Herbaceous_Wetlands
49911,209,2.0,2022,51199,Cantaloupes
49912,236,9.0,2022,51199,Dbl_Crop_WinWht/Sorghum


In [23]:
crops = crops[['count', 'year', 'county', 'Crop']]
crops.columns = ['Count', 'Year', 'County', 'Crop']
crop_replace_map = {
'Corn': 'Agriculture',
'Other_Hay/Non_Alfalfa': 'Agriculture',
'Soybeans': 'Agriculture',
'Alfalfa':'Agriculture',
'Sorghum':'Agriculture',
'Dbl_Crop_WinWht/Soybeans':'Agriculture',
'Winter_Wheat':'Agriculture',
'Dbl_Crop_WinWht/Corn':'Agriculture',
'Millet':'Agriculture',
'Dbl_Crop_Barley/Soybeans':'Agriculture',
'Rye':'Agriculture',
'Barley':'Agriculture',
'Dbl_Crop_Soybeans/Oats ':'Agriculture',
'Dbl_Crop_Barley/Corn':'Agriculture',
'Other_Crops':'Agriculture',
'Dbl_Crop_WinWht/Sorghum':'Agriculture',
'Oats':'Agriculture',
'Cotton':'Agriculture',
'Apples':'Agriculture',
'Pumpkins':'Agriculture',
'Tobacco':'Agriculture',
'Sunflower':'Agriculture',
'Christmas_Trees':'Agriculture',
'Triticale':'Agriculture',
'Grapes':'Agriculture',
'Sweet_Corn':'Agriculture',
'Peanuts':'Agriculture',
'Dry_Beans':'Agriculture',
'Dbl_Crop_Oats/Corn':'Agriculture',
'Potatoes':'Agriculture',
'Peaches':'Agriculture',
'Misc_Vegs_&_Fruits':'Agriculture',
'Dbl_Crop_Barley/Sorghum':'Agriculture',
'Sweet_Potatoes':'Agriculture',
'Squash':'Agriculture',
'Tomatoes':'Agriculture',
'Strawberries':'Agriculture',
'Broccoli':'Agriculture',
'Watermelons':'Agriculture',
'Other_Tree_Crops':'Agriculture',
'Canola':'Agriculture',
'Cantaloupes':'Agriculture',
'Cabbage':'Agriculture',
'Vetch':'Agriculture',
'Walnuts':'Agriculture',
'Rape_Seed':'Agriculture',
'Dbl_Crop_WinWht/Cotton':'Agriculture',
'Dbl_Crop_Corn/Soybeans':'Agriculture',
'Cherries':'Agriculture',
'Buckwheat':'Agriculture',
'Dbl_Crop_Soybeans/Cotton':'Agriculture',
'Switchgrass':'Agriculture',
'Cucumbers':'Agriculture',
'Cranberries':'Agriculture',
'Pop_or_Orn_Corn':'Agriculture',
'Radishes':'Agriculture',
'Greens':'Agriculture',
'Pears':'Agriculture',
'Peppers':'Agriculture',
'Pecans':'Agriculture',
'Peas':'Agriculture',
'Durum_Wheat':'Agriculture',
'Gourds':'Agriculture',
'Spring_Wheat':'Agriculture',
'Speltz':'Agriculture',
'Turnips':'Agriculture',
'Dbl_Crop_Soybeans/Oats':'Agriculture',
'Blueberries':'Agriculture',
'Lettuce':'Agriculture',
'Caneberries':'Agriculture',
'Asparagus':'Agriculture',
'Other_Small_Grains':'Agriculture',
'Plums':'Agriculture',

'Grassland/Pasture': 'Grassland',
'Sod/Grass_Seed': 'Grassland',
'Switchgrass':'Grassland', 
'Clover/Wildflowers':'Grassland',

'Mixed_Forest': 'Forest',
'Evergreen_Forest': 'Forest',
'Deciduous_Forest': 'Forest',

'Barren': 'Barren',
'Fallow/Idle_Cropland': 'Barren',

'Developed/High_Intensity': 'Developed',
'Developed/Med_Intensity': 'Developed',
'Developed/Low_Intensity': 'Developed',
'Developed/Open_Space': 'Developed',

'Open_Water': 'Open_Water',
'Shrubland': 'Shrubland',

'Woody_Wetlands':'Wetlands',
'Herbaceous_Wetlands':'Wetlands',

'Aquaculture':'Aquaculture'

}

crops['Landuse'] = crops['Crop'].map(crop_replace_map)
crops = crops.drop(['Crop'], axis=1)

In [24]:
crops

,Count,Year,County,Landuse
0,140068.0,2008,51001,Agriculture
1,229.0,2008,51001,Agriculture
2,120.0,2008,51001,Agriculture
3,128850.0,2008,51001,Agriculture
4,88.0,2008,51001,Agriculture
...,...,...,...,...
49909,48862.0,2022,51199,Wetlands
49910,8694.0,2022,51199,Wetlands
49911,2.0,2022,51199,Agriculture
49912,9.0,2022,51199,Agriculture


In [25]:
crop_counts = pd.DataFrame(crops.groupby(['Year', 'County', 'Landuse']).agg({'Count':'sum'})).reset_index()
crop_counts = crop_counts.pivot(index=['Year','County'], columns='Landuse', values='Count')
crop_counts = crop_counts.fillna(0)
crop_counts.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1410 entries, (2008, 51001) to (2022, 51199)
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Agriculture  1410 non-null   float64
 1   Aquaculture  1410 non-null   float64
 2   Barren       1410 non-null   float64
 3   Developed    1410 non-null   float64
 4   Forest       1410 non-null   float64
 5   Grassland    1410 non-null   float64
 6   Open_Water   1410 non-null   float64
 7   Shrubland    1410 non-null   float64
 8   Wetlands     1410 non-null   float64
dtypes: float64(9)
memory usage: 105.4 KB


In [26]:
crop_counts['Total'] = crop_counts.sum(axis=1, numeric_only=True)
cols = [col for col in crop_counts.columns if col != 'Total']
cols

['Agriculture',
 'Aquaculture',
 'Barren',
 'Developed',
 'Forest',
 'Grassland',
 'Open_Water',
 'Shrubland',
 'Wetlands']

In [27]:
for col in cols:
    crop_counts[col] = crop_counts[col]/crop_counts['Total']

In [28]:
crop_counts = crop_counts.drop(['Total'], axis=1)

In [29]:
crop_counts = crop_counts.reset_index()

In [30]:
income = pd.read_csv(path+'Income-county-all.csv')
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1380 entries, 0 to 1379
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Year    1380 non-null   int64
 1   County  1380 non-null   int64
 2   Income  1380 non-null   int64
dtypes: int64(3)
memory usage: 32.5 KB


In [31]:
final_df.County = final_df.County.astype(int)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1425 entries, 8 to 2372
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    1425 non-null   int64  
 1   County  1425 non-null   int64  
 2   DSCI    1425 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 44.5 KB


/var/folders/49/b4fjywl16vqb_14cy97qv8s40000gn/T/ipykernel_67137/4151909393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.County = final_df.County.astype(int)


In [32]:
crop_counts.Year = crop_counts.Year.astype(int)
crop_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 0 to 1409
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         1410 non-null   int64  
 1   County       1410 non-null   int64  
 2   Agriculture  1410 non-null   float64
 3   Aquaculture  1410 non-null   float64
 4   Barren       1410 non-null   float64
 5   Developed    1410 non-null   float64
 6   Forest       1410 non-null   float64
 7   Grassland    1410 non-null   float64
 8   Open_Water   1410 non-null   float64
 9   Shrubland    1410 non-null   float64
 10  Wetlands     1410 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 121.3 KB


In [33]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1425 entries, 8 to 2372
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    1425 non-null   int64  
 1   County  1425 non-null   int64  
 2   DSCI    1425 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 44.5 KB


In [34]:
df = pd.merge(final_df, income, on=['Year', 'County'], how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1425 entries, 0 to 1424
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    1425 non-null   int64  
 1   County  1425 non-null   int64  
 2   DSCI    1425 non-null   float64
 3   Income  975 non-null    float64
dtypes: float64(2), int64(2)
memory usage: 44.7 KB


In [35]:
df = pd.merge(df, crop_counts, on=['Year', 'County'], how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1425 entries, 0 to 1424
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         1425 non-null   int64  
 1   County       1425 non-null   int64  
 2   DSCI         1425 non-null   float64
 3   Income       975 non-null    float64
 4   Agriculture  1380 non-null   float64
 5   Aquaculture  1380 non-null   float64
 6   Barren       1380 non-null   float64
 7   Developed    1380 non-null   float64
 8   Forest       1380 non-null   float64
 9   Grassland    1380 non-null   float64
 10  Open_Water   1380 non-null   float64
 11  Shrubland    1380 non-null   float64
 12  Wetlands     1380 non-null   float64
dtypes: float64(11), int64(2)
memory usage: 144.9 KB


In [36]:
df

,Year,County,DSCI,Income,Agriculture,Aquaculture,Barren,Developed,Forest,Grassland,Open_Water,Shrubland,Wetlands
0,2008,51067,181.981132,31454.0,0.120512,0.0,0.000806,0.042941,0.663460,0.145732,0.026327,0.000216,0.000005
1,2009,51067,27.865385,30916.0,0.103555,0.0,0.001854,0.042546,0.662633,0.161879,0.027040,0.000486,0.000007
2,2010,51067,27.653846,31328.0,0.087572,0.0,0.001828,0.040608,0.660032,0.181759,0.027313,0.000887,0.000000
3,2011,51067,50.346154,33190.0,0.088977,0.0,0.002897,0.042279,0.672473,0.166330,0.026095,0.000917,0.000031
4,2012,51067,62.115385,34500.0,0.086611,0.0,0.003506,0.042493,0.673071,0.166485,0.026633,0.001170,0.000031
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420,2018,51093,16.884615,52662.0,0.286654,0.0,0.004196,0.075865,0.294034,0.008047,0.012790,0.112044,0.206370
1421,2019,51093,11.094340,55316.0,0.288809,0.0,0.000659,0.070008,0.373395,0.004695,0.014299,0.027932,0.220201
1422,2020,51093,0.826923,57943.0,0.284157,0.0,0.006378,0.070222,0.370450,0.003538,0.014950,0.025637,0.224667
1423,2021,51093,44.211538,62139.0,0.292600,0.0,0.000701,0.081463,0.354811,0.015733,0.014813,0.021250,0.218629


In [37]:
df = df[(df['Year'] >= 2008) & (df['Year'] <= 2022)]

In [38]:
for col in df.columns:
    print(col, len(df[df[col].isna()]))

Year 0
County 0
DSCI 0
Income 450
Agriculture 45
Aquaculture 45
Barren 45
Developed 45
Forest 45
Grassland 45
Open_Water 45
Shrubland 45
Wetlands 45


In [39]:
df[df.isna().any(axis=1)]

,Year,County,DSCI,Income,Agriculture,Aquaculture,Barren,Developed,Forest,Grassland,Open_Water,Shrubland,Wetlands
45,2008,51153,70.037736,NaN,0.088787,0.0,0.003993,0.310142,0.462579,0.075601,0.029458,0.006079,0.023361
46,2009,51153,37.557692,NaN,0.089595,0.0,0.007589,0.308793,0.451724,0.077501,0.030353,0.007197,0.027248
47,2010,51153,30.615385,NaN,0.128867,0.0,0.059944,0.176997,0.511754,0.083709,0.030280,0.000018,0.008431
48,2011,51153,47.788462,NaN,0.087966,0.0,0.005675,0.308881,0.482827,0.050295,0.029392,0.011502,0.023463
49,2012,51153,69.961538,NaN,0.084629,0.0,0.006825,0.309534,0.495311,0.047682,0.030073,0.008484,0.017462
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405,2018,51520,11.538462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1406,2019,51520,7.547170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407,2020,51520,1.923077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,2021,51520,11.730769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
co2 = pd.read_csv('individual-datasets/VA_CO2Emissions.csv')
co2 = co2[['Year', 'Total', 'Sector']]
co2

,Year,Total,Sector
0,1970,9.664151,Residential
1,1971,9.443978,Residential
2,1972,9.583187,Residential
3,1973,9.118182,Residential
4,1974,8.245722,Residential
...,...,...,...
307,2017,105.584974,Total
308,2018,110.056786,Total
309,2019,106.794626,Total
310,2020,98.175849,Total


In [41]:
co2 = co2.pivot_table(index=['Year'],columns='Sector', values='Total').reset_index()
co2.Year = co2.Year.astype(str)
co2 = co2.rename({'Total': 'TotalCO2'})
co2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 0 to 51
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            52 non-null     object 
 1   Commercial      52 non-null     float64
 2   Electic         52 non-null     float64
 3   Industrial      52 non-null     float64
 4   Residential     52 non-null     float64
 5   Total           52 non-null     float64
 6   Transportation  52 non-null     float64
dtypes: float64(6), object(1)
memory usage: 3.2+ KB


In [42]:
co2.Year = co2.Year.astype('int64')
co2 = co2[(co2['Year'] >= 2008) & (co2['Year'] <= 2022)]
co2 = co2[['Year', 'Total']]
co2 = co2.rename({'Total' : 'VACO2'}, axis=1)
co2

Sector,Year,VACO2
38,2008,116.763209
39,2009,105.907699
40,2010,111.886560
41,2011,102.519400
42,2012,98.562584
43,2013,105.767402
44,2014,107.442618
45,2015,107.620952
46,2016,110.065500
47,2017,105.584974


In [43]:
df = pd.merge(df, co2, on=['Year'], how='left')
df

,Year,County,DSCI,Income,Agriculture,Aquaculture,Barren,Developed,Forest,Grassland,Open_Water,Shrubland,Wetlands,VACO2
0,2008,51067,181.981132,31454.0,0.120512,0.0,0.000806,0.042941,0.663460,0.145732,0.026327,0.000216,0.000005,116.763209
1,2009,51067,27.865385,30916.0,0.103555,0.0,0.001854,0.042546,0.662633,0.161879,0.027040,0.000486,0.000007,105.907699
2,2010,51067,27.653846,31328.0,0.087572,0.0,0.001828,0.040608,0.660032,0.181759,0.027313,0.000887,0.000000,111.886560
3,2011,51067,50.346154,33190.0,0.088977,0.0,0.002897,0.042279,0.672473,0.166330,0.026095,0.000917,0.000031,102.519400
4,2012,51067,62.115385,34500.0,0.086611,0.0,0.003506,0.042493,0.673071,0.166485,0.026633,0.001170,0.000031,98.562584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420,2018,51093,16.884615,52662.0,0.286654,0.0,0.004196,0.075865,0.294034,0.008047,0.012790,0.112044,0.206370,110.056786
1421,2019,51093,11.094340,55316.0,0.288809,0.0,0.000659,0.070008,0.373395,0.004695,0.014299,0.027932,0.220201,106.794626
1422,2020,51093,0.826923,57943.0,0.284157,0.0,0.006378,0.070222,0.370450,0.003538,0.014950,0.025637,0.224667,98.175849
1423,2021,51093,44.211538,62139.0,0.292600,0.0,0.000701,0.081463,0.354811,0.015733,0.014813,0.021250,0.218629,97.953403


In [44]:
df.to_csv(path+'all-county-data-no-tmpprecip.csv', index=False)